<a id='ксодержанию'></a>
## Содержание
* [Импорт библиотек](#Импортбиблиотек)
* [Некоторые методы](#Некоторыеметоды)
* [Загрузка данных](#Загрузкаданных)
* [Модели](#Модели)
  + [raw_desc_tfidf_RF_specialization](#raw_desc_tfidf_RF_specialization)
    - [Моделируем](#Моделируемraw_desc_tfidf_RF_specialization)
    - [Отчёт](#Отчётraw_desc_tfidf_RF_specialization)
  + [raw_desc_tfidf_RF_prof_area](#raw_desc_tfidf_RF_prof_area)
    - [Моделируем](#Моделируемraw_desc_tfidf_RF_prof_area)
    - [Отчёт](#Отчётraw_desc_tfidf_RF_prof_area)
  + [lem_desc_tfidf_RF_prof_area](#lem_desc_tfidf_RF_prof_area)
    - [Моделируем](#Моделируемlem_desc_tfidf_RF_prof_area)
    - [Отчёт](#Отчётlem_desc_tfidf_RF_prof_area)
  + [lem_desc_tfidf_RF_specialization](#lem_desc_tfidf_RF_specialization)
    - [Моделируем](#Моделируемlem_desc_tfidf_RF_specialization)
    - [Отчёт](#Отчётlem_desc_tfidf_RF_specialization)
  + [raw_desc_tfidf_LR_prof_area_with_len](#raw_desc_tfidf_LR_prof_area_with_len)
    - [Моделируем](#Моделируемraw_desc_tfidf_LR_prof_area_with_len)
    - [Отчёт](#Отчётraw_desc_tfidf_LR_prof_area_with_len)
  + [lem_desc_tfidf_LR_prof_area_14000](#lem_desc_tfidf_LR_prof_area_14000)
    - [Моделируем](#Моделируемlem_desc_tfidf_LR_prof_area_14000)
    - [Отчёт](#Отчётlem_desc_tfidf_LR_prof_area_14000)

<a id='Импортбиблиотек'></a> 
# Импорт библиотек
[к содержанию](#ксодержанию)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -U colabcode
import colabcode
colabcode.ColabCode(
    port=12000,
    authtoken='21eNVw6XbU2fjExgUWxa47wNx8x_6AfyfxMFYUmSWFLxW7b72',
    lab=True
)

Mounted at /content/drive
     |████████████████████████████████| 8.3 MB 5.5 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
     |████████████████████████████████| 745 kB 37.7 MB/s 
     |████████████████████████████████| 58 kB 6.8 MB/s 
     |████████████████████████████████| 428 kB 35.4 MB/s 
     |████████████████████████████████| 396 kB 34.1 MB/s 
     |████████████████████████████████| 54 kB 3.9 MB/s 
     |████████████████████████████████| 78 kB 7.7 MB/s 
     |████████████████████████████████| 129 kB 60.2 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 125 kB 65.4 MB/s 
     |████████████████████████████████| 124 kB 75.2 MB/s 
     |████████████████████████████████| 124 kB 74.0 MB/s 
     |████████████████████████████████| 122 kB 61.1 MB/s 
     |████████████████████████████████| 122 kB 61.6 MB/s 
     |████████████████████████████████| 122 kB 61.3 MB/s 
     |████████████████████████████████| 122 kB 57.5 MB/s 

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from tabulate import tabulate
import joblib
import numpy as np
from sklearn.linear_model import LogisticRegression
import pandas as pd

<a id='Некоторыеметоды'></a> 
# Некоторые методы
[к содержанию](#ксодержанию)

In [2]:
def salary_process(df: pd.DataFrame):
  """
  Создаём метод, который преобразовывает столбец с з/п
  Аргумент:
  df - датасет со всем массивом данных
  """
  # Переводим з/п в рубли по курсу на 09.12.2021
  currency = {'RUR': 1, 'BYR': 29.04, 'KZT': 0.17, 'UZS': 0.0068, 'UAH': 2.72, 'USD': 73.66, 'KGS': 0.87, 'EUR': 83.32, 'AZN': 43.33, 'GEL': 23.73}
  # Переводим з/п в нерублёвых валютах в рубли
  for curr in df['salary_currency'].unique():
      df['salary_from'][df['salary_currency'] == curr] = df['salary_from'][df['salary_currency'] == curr].apply(lambda x: x*currency[curr])
  # Нормируем на НДФЛ для каждой страны
  df.loc[(df['salary_gross'] == False) & ((df['country_name'] == 'Россия') | (df['country_name'] == 'Беларусь')), 'salary_from'] = df['salary_from'] / 0.87
  df.loc[(df['salary_gross'] == False) & ((df['country_name'] == 'Казахстан') | (df['country_name'] == 'Кыргызстан')), 'salary_from'] = df['salary_from'] / 0.9
  df.loc[(df['salary_gross'] == False) & (df['country_name'] == 'Украина'), 'salary_from'] = df['salary_from'] / 0.82
  df.loc[(df['salary_gross'] == False) & (df['country_name'] == 'Грузия'), 'salary_from'] = df['salary_from'] / 0.8

In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
def report(df: pd.DataFrame, y_test: pd.DataFrame, y_pred: np.array, col: str):
  """
  Создаём метод, который создаёт регрессионный репорт
  Аргументы:
  df - датасет со всем массивом данных
  y_test - датасет с тестовой выборкой
  y_pred - датасет с предсказанными сферами/специализациями
  col - название столбца, по которому происходит группировка: либо 'prof_area', либо 'specialization'
  Возвращает:

  """
  salary_groupped_prof_area = df[[col, 'salary_from']].groupby([col]).mean()
  salary_groupped_prof_area_std = df[[col, 'salary_from']].groupby([col]).std()
  salary_pred = salary_groupped_prof_area.loc[y_pred]
  salary_test = df.loc[y_test.index, 'salary_from']
  salary = pd.DataFrame()
  salary['y_pred'] = y_pred
  salary['test'] =  list(salary_test)
  salary['pred'] = list(salary_pred['salary_from'])
  salary['true'] = list(df.loc[y_test.index, col])
  salary = salary.dropna()
  regression_report = []
  for area in salary['true'].unique():
    regression_report.append([round(mean_absolute_error(salary['pred'][salary['true'] == area], salary['test'][salary['true'] == area]), 1),
    round(float(salary_groupped_prof_area.loc[area]), 1), round(float(salary_groupped_prof_area_std.loc[area]), 1)])
  return regression_report, salary

In [4]:
def fill_prof_area(prof_areas, row:  list):
  """
  Создаём метод, который заполняет словарь prof_area сферами и частотой их встречания в вакансиях
  Аргумент:
  row - строка в датафрейме
  """
  for el in row:
    if el in prof_areas:
      prof_areas[el] += 1
    else:
      prof_areas[el] = 1

def decrease_specializations(x: str):
  """
  Создаём метод, которые вместо наименее популярных специализаций пишет "Другое"
  Аргумент:
  х - специализация
  Возвращает:
  х - специализация
  """
  try:
    if specializations[x]/len(df) < 0.01:
      return 'Другое'
    else:
      return x
  except:
    return x

<a id='Загрузкаданных'></a>
# Загрузка данных
[к содержанию](#ксодержанию)

In [5]:
!gdown --id 1A-59GMNuJWbrbCc17zvBPkOycSdCBdCA

Downloading...
From: https://drive.google.com/uc?id=1A-59GMNuJWbrbCc17zvBPkOycSdCBdCA
To: /content/vacancies_lemmatized.csv
100% 902M/902M [00:06<00:00, 134MB/s]


In [6]:
df = pd.read_csv('vacancies_lemmatized.csv')
df = df[df['language'] == 'ru']
salary_process(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1021: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


<a id='Модели'></a>
# Модели
[к содержанию](#ксодержанию)

<a id='raw_desc_tfidf_RF_specialization'></a>
## raw_desc_tfidf_RF_specialization
[к содержанию](#ксодержанию)

<a id='Моделируемraw_desc_tfidf_RF_specialization'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [8]:
df = df[['raw_description', 'specialization', 'specializations_name']].dropna()

In [9]:
specializations = dict()
df['specializations_name'].apply(lambda x: fill_prof_area(specializations, eval(x)));

In [10]:
df['specialization'] = df['specialization'].apply(decrease_specializations)

In [11]:
X = df[['raw_description', 'specialization']]
X_train, X_test, y_train, y_test = train_test_split(X[['raw_description']], X['specialization'], test_size=0.2, random_state=42)

In [12]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words={'russian', 'english'}, max_features=4000)
tfidf.fit(X_train['raw_description'])
features = tfidf.transform(X_train['raw_description'])

In [ ]:
# Сохраняем tfidf.vocabulary_
import pickle
pickle.dump(tfidf.vocabulary_,open("raw_desc_tfidf.pkl","wb"))

In [13]:
del X_train

In [14]:
model = RandomForestClassifier()
model.fit(features, y_train)

RandomForestClassifier()

In [15]:
del features

In [16]:
features_test = tfidf.transform(X_test['raw_description'])

In [17]:
y_pred = model.predict(features_test)

In [18]:
# Сохраняем модель и y_pred
joblib.dump(model, 'raw_desc_tfidf_RF_specialization.pkl', compress=8)
joblib.dump(y_pred, 'y_pred_raw_desc_tfidf_RF_specialization.pkl')

['y_pred_raw_desc_tfidf_RF_specialization.pkl']

<a id='Отчётraw_desc_tfidf_RF_specialization'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [ ]:
df = pd.read_csv('vacancies_lemmatized.csv')
df = df[df['language'] == 'ru']
df['specialization'] = df['specialization'].apply(decrease_specializations)

In [ ]:
regression_report, salary = report(df, y_test, y_pred, 'specialization')

In [ ]:
report = pd.DataFrame(metrics.classification_report(y_test, y_pred, output_dict=True)).T
report.style.format(formatter='{:,.2f}').background_gradient()

In [ ]:
report = pd.DataFrame(regression_report)
report.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
report.index = salary['true'].unique()
report.style.format(formatter='{:,.1f}').background_gradient()

<a id='raw_desc_tfidf_RF_prof_area'></a>
## raw_desc_tfidf_RF_prof_area
[к содержанию](#ксодержанию)

<a id='Моделируемraw_desc_tfidf_RF_prof_area'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [21]:
df = pd.read_csv('vacancies_lemmatized.csv')
df = df[df['language'] == 'ru']
salary_process(df)
X = df[['raw_description', 'prof_area']]
X_train, X_test, y_train, y_test = train_test_split(X[['raw_description']], X['prof_area'], test_size=0.2, random_state=42)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1021: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [22]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words={'russian', 'english'}, max_features=4000)
tfidf.fit(X_train['raw_description'])
features = tfidf.transform(X_train['raw_description'])

In [ ]:
# Сохраняем tfidf.vocabulary_
import pickle
pickle.dump(tfidf.vocabulary_,open("raw_desc_tfidf.pkl","wb"))

In [23]:
del X_train

In [24]:
model = RandomForestClassifier()
model.fit(features, y_train)

RandomForestClassifier()

In [25]:
del features

In [26]:
features_test = tfidf.transform(X_test['raw_description'])

In [27]:
y_pred = model.predict(features_test)

In [28]:
# Сохраняем модель и y_pred
joblib.dump(model, 'raw_desc_tfidf_RF_prof_area.pkl', compress=8)
joblib.dump(y_pred, 'y_pred_raw_desc_tfidf_RF_prof_area.pkl')

['y_pred_raw_desc_tfidf_RF_prof_area.pkl']

<a id='Отчётraw_desc_tfidf_RF_prof_area'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [ ]:
regression_report, salary = report(df, y_test, y_pred, 'prof_area')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
report_raw = pd.DataFrame(metrics.classification_report(y_test, y_pred, output_dict=True)).T
report_raw.style.format(formatter='{:,.2f}').background_gradient()

,precision,recall,f1-score,support
Автомобильный бизнес,0.79,0.45,0.57,535.00
Административный персонал,0.56,0.54,0.55,"1,389.00"
"Банки, инвестиции, лизинг",0.89,0.81,0.85,767.00
Безопасность,0.85,0.61,0.71,420.00
"Бухгалтерия, управленческий учет, финансы предприятия",0.67,0.76,0.71,798.00
Высший менеджмент,0.79,0.19,0.31,137.00
"Государственная служба, некоммерческие организации",0.83,0.53,0.65,123.00
Добыча сырья,0.85,0.33,0.47,289.00
Домашний персонал,0.81,0.14,0.24,94.00
Закупки,0.84,0.20,0.33,157.00


In [ ]:
regression_report_raw = pd.DataFrame(regression_report)
regression_report_raw.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
regression_report_raw.index = salary['true'].unique()
regression_report_raw.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
"Туризм, гостиницы, рестораны","19,250.7","36,760.9","31,968.7"
"Медицина, фармацевтика","19,482.1","49,268.2","27,663.4"
"Строительство, недвижимость","30,729.2","69,788.8","45,062.1"
"Транспорт, логистика","23,882.6","56,083.3","36,173.1"
Добыча сырья,"45,211.8","102,632.2","50,338.6"
Продажи,"14,920.0","36,663.7","94,863.2"
"Банки, инвестиции, лизинг","15,644.9","43,680.9","23,496.3"
Административный персонал,"15,005.0","34,107.2","19,094.3"
"Информационные технологии, интернет, телеком","27,698.1","50,753.6","47,597.3"
"Маркетинг, реклама, PR","14,576.4","35,473.8","18,644.8"


<a id='lem_desc_tfidf_RF_prof_area'></a>
# lem_desc_tfidf_RF_prof_area
[к содержанию](#ксодержанию)

<a id='Моделируемlem_desc_tfidf_RF_prof_area'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [29]:
X = df[['lem_raw_desc', 'prof_area']]
X_train, X_test, y_train, y_test = train_test_split(X[['lem_raw_desc']], X['prof_area'], test_size=0.2, random_state=42)

In [30]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words={'russian', 'english'}, max_features=4000)
tfidf.fit(X_train['lem_raw_desc'])
features = tfidf.transform(X_train['lem_raw_desc'])

In [ ]:
# Сохраняем tfidf.vocabulary_
import pickle
pickle.dump(tfidf.vocabulary_,open("lem_desc_tfidf.pkl","wb"))

In [31]:
del X_train

In [32]:
model = RandomForestClassifier()
model.fit(features, y_train)

RandomForestClassifier()

In [33]:
del features

In [34]:
features_test = tfidf.transform(X_test['lem_raw_desc'])

In [35]:
y_pred = model.predict(features_test)

In [45]:
# Сохраняем модель и y_pred
joblib.dump(model, 'lem_desc_tfidf_RF_prof_area.pkl', compress=8)
joblib.dump(y_pred, 'y_pred_lem_desc_tfidf_RF_prof_area.pkl')

['y_pred_lem_desc_tfidf_RF_prof_area.pkl']

<a id='Отчётlem_desc_tfidf_RF_prof_area'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [ ]:
report = pd.DataFrame(metrics.classification_report(y_test, y_pred, output_dict=True)).T
report.style.format(formatter='{:,.2f}').background_gradient()

,precision,recall,f1-score,support
Автомобильный бизнес,0.80,0.45,0.57,535.00
Административный персонал,0.56,0.53,0.55,"1,389.00"
"Банки, инвестиции, лизинг",0.90,0.82,0.86,767.00
Безопасность,0.88,0.64,0.74,420.00
"Бухгалтерия, управленческий учет, финансы предприятия",0.71,0.78,0.74,798.00
Высший менеджмент,0.81,0.19,0.31,137.00
"Государственная служба, некоммерческие организации",0.88,0.50,0.64,123.00
Добыча сырья,0.86,0.36,0.51,289.00
Домашний персонал,0.87,0.14,0.24,94.00
Закупки,0.84,0.20,0.32,157.00


In [ ]:
regression_report, salary = report(df, y_test, y_pred, 'prof_area')

In [ ]:
regression_report_lem = pd.DataFrame(regression_report)
regression_report_lem.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
regression_report_lem.index = salary['true'].unique()
regression_report_lem.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
"Туризм, гостиницы, рестораны","19,349.8","36,760.9","31,968.7"
"Медицина, фармацевтика","19,590.7","49,268.2","27,663.4"
"Строительство, недвижимость","30,613.5","69,788.8","45,062.1"
"Транспорт, логистика","23,834.9","56,083.3","36,173.1"
Добыча сырья,"43,415.8","102,632.2","50,338.6"
Продажи,"14,925.3","36,663.7","94,863.2"
"Банки, инвестиции, лизинг","15,679.4","43,680.9","23,496.3"
Административный персонал,"15,087.0","34,107.2","19,094.3"
"Информационные технологии, интернет, телеком","27,811.5","50,753.6","47,597.3"
"Маркетинг, реклама, PR","14,365.3","35,473.8","18,644.8"


<a id='lem_desc_tfidf_RF_specialization'></a>
# lem_desc_tfidf_RF_specialization
[к содержанию](#ксодержанию)

<a id='Моделируемlem_desc_tfidf_RF_specialization'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [38]:
df = df[['lem_raw_desc', 'specialization', 'specializations_name']].dropna()

In [39]:
specializations = dict()
df['specializations_name'].apply(lambda x: fill_prof_area(specializations, eval(x)));

In [40]:
df['specialization'] = df['specialization'].apply(decrease_specializations)

In [41]:
X = df[['lem_raw_desc', 'specialization']]
X_train, X_test, y_train, y_test = train_test_split(X[['lem_raw_desc']], X['specialization'], test_size=0.2, random_state=42)

In [42]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words={'russian', 'english'}, max_features=4000)
tfidf.fit(X_train['lem_raw_desc'])
features = tfidf.transform(X_train['lem_raw_desc'])

In [ ]:
# Сохраняем tfidf.vocabulary_
import pickle
pickle.dump(tfidf.vocabulary_,open("lem_desc_tfidf.pkl","wb"))

In [43]:
del X_train

In [46]:
model = RandomForestClassifier()
model.fit(features, y_train)

RandomForestClassifier()

In [47]:
del features

In [49]:
features_test = tfidf.transform(X_test['lem_raw_desc'])

In [50]:
y_pred = model.predict(features_test)

In [52]:
# Сохраняем модель и y_pred
joblib.dump(model, 'lem_desc_tfidf_RF_specialization.pkl', compress=8)
joblib.dump(y_pred, 'y_pred_lem_desc_tfidf_RF_specialization.pkl')

['y_pred_lem_desc_tfidf_RF_specialization.pkl']

<a id='Отчётlem_desc_tfidf_RF_specialization'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [ ]:
df = pd.read_csv('vacancies_lemmatized.csv')
df = df[df['language'] == 'ru']
df['specialization'] = df['specialization'].apply(decrease_specializations)

In [ ]:
regression_report, salary = report(df, y_test, y_pred, 'specialization')

In [ ]:
report_lem = pd.DataFrame(metrics.classification_report(y_test, y_pred, output_dict=True)).T
report_lem.style.format(formatter='{:,.2f}').background_gradient()

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
"FMCG, Товары народного потребления",0.81,0.51,0.63,393.00
Автоперевозки,0.60,0.58,0.59,563.00
Бухгалтер,0.73,0.59,0.65,527.00
Ввод данных,0.88,0.24,0.38,172.00
Водитель,0.68,0.44,0.54,411.00
Грузчик,0.65,0.61,0.63,433.00
Делопроизводство,0.84,0.11,0.19,250.00
Дистрибуция,0.82,0.35,0.49,51.00
Другое,0.60,0.94,0.73,"9,335.00"
Инженер,0.96,0.15,0.26,156.00


In [ ]:
report = pd.DataFrame(regression_report)
report.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
report.index = salary['true'].unique()
report.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
Управление проектами,"56,776.8","181,628.5","821,148.3"
Разнорабочий,"31,914.6","52,019.4","70,398.8"
Другое,"77,155.5","77,449.7","354,172.0"
Розничная торговля,"50,765.6","47,827.8","304,353.6"
Машиностроение,"51,514.5","60,544.0","92,169.2"
"Официант, Бармен","58,300.5","53,049.8","211,793.5"
Бухгалтер,"118,568.6","103,655.5","554,758.5"
"Начальный уровень, Мало опыта","49,722.4","37,974.9","105,188.2"
Кладовщик,"34,748.6","45,422.2","106,330.6"
Прямые продажи,"56,661.6","99,982.1","422,951.5"


In [ ]:
report_lem.to_csv('lem_desc_tfidf_RF_specialization.csv')
report.to_csv('lem_desc_tfidf_RF_specialization.csv')

<a id='raw_desc_tfidf_LR_prof_area_with_len'></a>
# raw_desc_tfidf_LR_prof_area_with_len
[к содержанию](#ксодержанию)

<a id='Моделируемraw_desc_tfidf_LR_prof_area_with_len'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [7]:
X = df[['len_description_responsibilities', 'len_description_requirements', 'len_description_conditions', 'raw_description', 'prof_area']]
X_train, X_test, y_train, y_test = train_test_split(X[['raw_description', 'len_description_responsibilities', 'len_description_requirements', 'len_description_conditions']], X['prof_area'], test_size=0.2, random_state=42)

In [8]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words={'russian', 'english'}, max_features=4000)
tfidf.fit(X_train['raw_description'])
features = tfidf.transform(X_train['raw_description'])

In [ ]:
# Сохраняем tfidf.vocabulary_
import pickle
pickle.dump(tfidf.vocabulary_,open("raw_desc_tfidf.pkl","wb"))

In [9]:
# Добавляем фичи с длинами массивов
feat = ['len_description_responsibilities', 'len_description_requirements', 'len_description_conditions']
features = pd.DataFrame(features.toarray())
for feature in feat:
  features[feature] = X_train[feature]
  features[feature]= features[feature].fillna(features[feature].mode().values[0])
features = features.to_numpy()

In [10]:
del X_train

In [11]:
model = LogisticRegression()
model.fit(features, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [12]:
del features

In [13]:
features_test = tfidf.transform(X_test['raw_description'])

In [14]:
# Добавляем фичи с длинами массивов
feat = ['len_description_responsibilities', 'len_description_requirements', 'len_description_conditions']
features_test = pd.DataFrame(features_test.toarray())
for feature in feat:
  features_test[feature] = X_test[feature]
  features_test[feature]= features_test[feature].fillna(features_test[feature].mode().values[0])
features_test = features_test.to_numpy()

In [15]:
y_pred = model.predict(features_test)

In [16]:
# Сохраняем модель и y_pred
joblib.dump(model, 'raw_desc_tfidf_LR_prof_area_with_len.pkl', compress=8)
joblib.dump(y_pred, 'y_pred_raw_desc_tfidf_LR_prof_area_with_len.pkl')

['y_pred_raw_desc_tfidf_LR_prof_area_with_len.pkl']

<a id='Отчётraw_desc_tfidf_LR_prof_area_with_len'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [ ]:
report_lem = pd.DataFrame(metrics.classification_report(y_test, y_pred, output_dict=True)).T
report_lem.style.format(formatter='{:,.2f}').background_gradient()

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
Автомобильный бизнес,0.80,0.27,0.40,535.00
Административный персонал,0.39,0.57,0.46,"1,389.00"
"Банки, инвестиции, лизинг",0.72,0.71,0.72,767.00
Безопасность,0.82,0.32,0.46,420.00
"Бухгалтерия, управленческий учет, финансы предприятия",0.61,0.72,0.66,798.00
Высший менеджмент,1.00,0.01,0.03,137.00
"Государственная служба, некоммерческие организации",0.89,0.34,0.49,123.00
Добыча сырья,1.00,0.03,0.06,289.00
Домашний персонал,0.00,0.00,0.00,94.00
Закупки,0.56,0.03,0.06,157.00


In [ ]:
regression_report, salary = report(df, y_test, y_pred, 'prof_area')

In [ ]:
report = pd.DataFrame(regression_report)
report.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
report.index = salary['true'].unique()
report.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
"Туризм, гостиницы, рестораны","19,269.3","36,760.9","31,968.7"
"Медицина, фармацевтика","19,352.5","49,268.2","27,663.4"
"Строительство, недвижимость","31,551.4","69,788.8","45,062.1"
"Транспорт, логистика","23,423.4","56,083.3","36,173.1"
Добыча сырья,"47,630.7","102,632.2","50,338.6"
Продажи,"14,841.4","36,663.7","94,863.2"
"Банки, инвестиции, лизинг","15,669.1","43,680.9","23,496.3"
Административный персонал,"15,075.9","34,107.2","19,094.3"
"Информационные технологии, интернет, телеком","26,961.8","50,753.6","47,597.3"
"Маркетинг, реклама, PR","14,759.9","35,473.8","18,644.8"


In [ ]:
report.to_csv('regression_report_raw_desc_tfidf_LR_prof_area_with_len.csv')

<a id='lem_desc_tfidf_LR_prof_area_14000'></a>
# lem_desc_tfidf_LR_prof_area_14000
[к содержанию](#ксодержанию)

<a id='Моделируемlem_desc_tfidf_LR_prof_area_14000'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [17]:
X = df[['lem_raw_desc', 'prof_area']]
X_train, X_test, y_train, y_test = train_test_split(X[['lem_raw_desc']], X['prof_area'], test_size=0.2, random_state=42)

In [18]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words={'russian', 'english'}, max_features=14000)
tfidf.fit(X_train['lem_raw_desc'])
features = tfidf.transform(X_train['lem_raw_desc'])

In [ ]:
# Сохраняем tfidf.vocabulary_
import pickle
pickle.dump(tfidf.vocabulary_,open("lem_desc_tfidf.pkl","wb"))

In [19]:
del X_train

In [20]:
model = LogisticRegression()
model.fit(features, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [21]:
del features

In [22]:
features_test = tfidf.transform(X_test['lem_raw_desc'])

In [23]:
y_pred = model.predict(features_test)

In [24]:
# Сохраняем модель и y_pred
joblib.dump(model, 'lem_desc_tfidf_LR_prof_area_14000.pkl', compress=8)
joblib.dump(y_pred, 'y_pred_lem_desc_tfidf_LR_prof_area_14000.pkl')

['y_pred_lem_desc_tfidf_LR_prof_area_14000.pkl']

<a id='Отчётlem_desc_tfidf_LR_prof_area_14000'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [ ]:
report_lem_LR = pd.DataFrame(metrics.classification_report(y_test, y_pred, output_dict=True)).T
report_lem_LR.style.format(formatter='{:,.2f}').background_gradient()

,precision,recall,f1-score,support
Автомобильный бизнес,0.77,0.49,0.60,535.00
Административный персонал,0.55,0.63,0.59,"1,389.00"
"Банки, инвестиции, лизинг",0.87,0.82,0.85,767.00
Безопасность,0.84,0.72,0.78,420.00
"Бухгалтерия, управленческий учет, финансы предприятия",0.81,0.79,0.80,798.00
Высший менеджмент,0.65,0.23,0.34,137.00
"Государственная служба, некоммерческие организации",0.80,0.53,0.64,123.00
Добыча сырья,0.77,0.46,0.58,289.00
Домашний персонал,0.77,0.29,0.42,94.00
Закупки,0.63,0.36,0.46,157.00


In [ ]:
regression_report, salary = report(df, y_test, y_pred, 'prof_area')

In [ ]:
regression_report_lem_LR = pd.DataFrame(regression_report)
regression_report_lem_LR.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
regression_report_lem_LR.index = salary['true'].unique()
regression_report_lem_LR.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
"Туризм, гостиницы, рестораны","18,748.7","36,760.9","31,968.7"
"Медицина, фармацевтика","19,400.5","49,268.2","27,663.4"
"Строительство, недвижимость","31,204.4","69,788.8","45,062.1"
"Транспорт, логистика","23,742.2","56,083.3","36,173.1"
Добыча сырья,"41,838.3","102,632.2","50,338.6"
Продажи,"14,803.2","36,663.7","94,863.2"
"Банки, инвестиции, лизинг","15,571.7","43,680.9","23,496.3"
Административный персонал,"14,662.8","34,107.2","19,094.3"
"Информационные технологии, интернет, телеком","27,277.0","50,753.6","47,597.3"
"Маркетинг, реклама, PR","13,271.3","35,473.8","18,644.8"
